In [ ]:
from pathlib import Path

starter_script = """
# 🧠 Starter Script: EEG Mood → Multi-instrument MIDI → Audio Output using pretty_midi + pyfluidsynth

import pretty_midi
import fluidsynth
import time
import os

# --- Step 1: Create a new PrettyMIDI object
midi = pretty_midi.PrettyMIDI()

# --- Step 2: Add instruments
# 🎹 Pad (channel 0)
pad = pretty_midi.Instrument(program=89)  # GM program 89 = Pad 1 (New Age)
pad.notes.append(pretty_midi.Note(velocity=80, pitch=60, start=0.0, end=2.0))
pad.notes.append(pretty_midi.Note(velocity=80, pitch=64, start=2.0, end=4.0))
pad.notes.append(pretty_midi.Note(velocity=80, pitch=67, start=4.0, end=6.0))

# 🎸 Bass (channel 1)
bass = pretty_midi.Instrument(program=33)  # Fingered Bass
bass.notes.append(pretty_midi.Note(velocity=100, pitch=43, start=0.0, end=1.5))
bass.notes.append(pretty_midi.Note(velocity=100, pitch=45, start=2.0, end=3.5))

# 🥁 Drums (channel 9)
drums = pretty_midi.Instrument(program=0, is_drum=True)
drums.notes.append(pretty_midi.Note(velocity=127, pitch=36, start=0.0, end=0.1))  # Kick
drums.notes.append(pretty_midi.Note(velocity=100, pitch=38, start=1.0, end=1.1))  # Snare
drums.notes.append(pretty_midi.Note(velocity=100, pitch=42, start=2.0, end=2.1))  # Hi-hat

# Add instruments to MIDI
midi.instruments += [pad, bass, drums]

# --- Step 3: Save to file
midi_path = "mood_music.mid"
midi.write(midi_path)
print(f"Saved MIDI file to {midi_path}")

# --- Step 4: Play it back using pyfluidsynth
# Load a soundfont (make sure you have one)
soundfont_path = "FluidR3_GM.sf2"  # Replace with the path to your .sf2 file
assert os.path.exists(soundfont_path), f"SoundFont not found: {soundfont_path}"

# Initialize FluidSynth
fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")
sfid = fs.sfload(soundfont_path)
fs.program_select(0, sfid, 0, 89)  # Pad
fs.program_select(1, sfid, 0, 33)  # Bass
fs.program_select(9, sfid, 128, 0)  # Drums on channel 9 (bank 128)

# Playback from PrettyMIDI
for instrument in midi.instruments:
    for note in instrument.notes:
        channel = 9 if instrument.is_drum else midi.instruments.index(instrument)
        fs.noteon(channel, note.pitch, note.velocity)
        time.sleep(note.end - note.start)
        fs.noteoff(channel, note.pitch)

# Shutdown synth
fs.delete()
"""

# Save the script to a file
starter_file_path = Path("/mnt/data/mood_midi_starter.py")
starter_file_path.write_text(starter_script)

starter_file_path.name
